In [1]:
import torch
import os
import pandas as pd
import scipy
import torchvision
from torch import optim,nn
from torch.utils.data import Dataset
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


In [3]:
class AE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encoder_hidden_layer = nn.Linear(
            in_features=kwargs["input_shape"], out_features=128
        )
        self.encoder_output_layer = nn.Linear(
            in_features=128, out_features=128
        )
        self.decoder_hidden_layer = nn.Linear(
            in_features=128, out_features=128
        )
        self.decoder_output_layer = nn.Linear(
            in_features=128, out_features=kwargs["input_shape"]
        )

    def forward(self, features):
        activation = self.encoder_hidden_layer(features)
        activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        code = torch.relu(code)
        activation = self.decoder_hidden_layer(code)
        activation = torch.relu(activation)
        activation = self.decoder_output_layer(activation)
        reconstructed = torch.relu(activation)
        return reconstructed



In [4]:

class StanfordCars(Dataset):

    def __init__(self, root: str, split: str = "train", transform=None, target_transform=None):
        self.transform = transform
        self.target_transform = target_transform
        self._split = split
        self._base_folder = root

        if self._split == "train":
            self._images_base_path = self._base_folder + "/train/"
        else:
            self._images_base_path = self._base_folder + "/test/"

        annotation = pd.DataFrame({"Filename":os.listdir(self._images_base_path)})

        self._samples = [
            (
                str(self._images_base_path + annot),
            )
            for annot in annotation["Filename"]
        ]


    def __len__(self) -> int:
        return len(self._samples)

    def __getitem__(self, idx: int):
        """Returns pil_image and class_id for given index"""
        image_path = self._samples[idx]
        target = 0
        
        pil_image = Image.open(image_path[0]).convert("RGB")

        if self.transform is not None:
            pil_image = self.transform(pil_image)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return (pil_image, target)


img_size = 288
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((img_size,img_size)),
    #T.RandomResizedCrop(image_size), # data augmentation
    # T.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor()])

train_dataset = StanfordCars(root="/data/students/louis/standfordcars/standfordcars",split ="test",transform=transform)

test_dataset = StanfordCars(root="/data/students/louis/standfordcars/standfordcars",split ="train",transform=transform)

train_batch = 128
train_loader = torch.utils.data.DataLoader( train_dataset, batch_size=train_batch, shuffle=True, num_workers=4, pin_memory=True)

test_loader = torch.utils.data.DataLoader(  test_dataset, batch_size=32, shuffle=False, num_workers=4)


In [ ]:

#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu",0)
print(device)
# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = AE(input_shape=img_size*img_size*3).to(device)

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=1e-2)

# mean-squared error loss
criterion = nn.MSELoss()


epochs = 50
restored_imgs = []
losses = []
for epoch in range(epochs):
    loss = 0
    for batch_features, _ in train_loader:
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
        batch_features = batch_features.view(-1, img_size*img_size*3).to(device)
    
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()
        
        # compute reconstructions
        outputs = model(batch_features)
        
        # compute training reconstruction loss
        train_loss = criterion(outputs, batch_features)
        
        # compute accumulated gradients
        train_loss.backward()
        
        # perform parameter update based on current gradients
        optimizer.step()
        
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(train_loader)
    losses.append(loss)
    restored_imgs.append((epochs, batch_features, outputs))
 
    # display the epoch training loss
    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))


In [ ]:

dataiter = iter(test_loader)
images, labels = dataiter.next()

def imshowo(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# print images
imshowo(torchvision.utils.make_grid(images))


outputs = model(images.view(-1, img_size*img_size*3).to(device))
rep = torch.Tensor.cpu(outputs).detach()
# plt.imshow(rep.view(4*288,8*288,3))
# plt.show()
imshowo(torchvision.utils.make_grid(rep.view(32,3,288,288)))